<a href="https://colab.research.google.com/github/alberto2020china/Topic_Modelling/blob/main/1_ac_cluster%26topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1- Libraries and dependencies



In [1]:
!pip install imblearn
!pip install transformers
!pip install umap-learn
!pip install hdbscan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.3 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=4e23a57129437f54139a1b81c95215815e1c3c1cc9e10b5ed73fa8e543724cda
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039276 sha256=ceb8972941fdd7b840f5c97b1d558b2fdd1a844dae3e10d4f00488eae6c0af75
  S

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string

#import contractions # I'm you'r etc
import nltk
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer

from sklearn.model_selection import train_test_split


from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential


from tqdm import tqdm # printing the status bar
from bs4 import BeautifulSoup

import requests
from io import BytesIO
from zipfile import ZipFile

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)


from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

#2-Import Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/My Drive/Omdena/DataInn/complete_documents_metadata_summary.csv',encoding='windows-1252')

In [5]:
df

,Title,Organization,Keywords,Summary
0,Action Coalition on Innovation and Technology ...,Various,"Gender Equality, Technology, Inclusive Design",Advocates for integrating gender equality in d...
1,Chinese Academy of Sciences,Chinese Academy of Sciences,"Big Data, Sustainability, CBAS",Discusses the role of big data in sustainable ...
2,International Federation of Library Associatio...,IFLA,"Libraries, Information Access, SDGs",Emphasizes libraries' role in achieving the Su...
3,Global Partners Digital input to GDC,Global Partners Digital,"Digital Inclusion, Human Rights, Internet Policy",Addresses digital rights and internet governan...
4,International Chamber of Commerce,International Chamber of Commerce,"Internet Connectivity, AI Regulation, Human Ri...","Focuses on the Global Digital Compact, coverin..."
5,Alliance for Universal Digital Rights,Alliance for Universal Digital Rights,"Digital Technologies, Universal Access, Privacy","Advocates for digital rights as human rights, ..."
6,Equity 2030: Building Equitable Future - Submi...,Equity 2030,"Gender Equity, Technology, Inclusion",Discusses strategies to achieve gender equity ...
7,GDC Stimson Center: Rethinking Global Cooperation,Stimson Center,"Global Cooperation, Digital Policy, Innovation",Explores innovative approaches to global coope...
8,GDC submission EuroSSIG,EuroSSIG,"Internet Governance, Education, Stakeholders",Discusses the importance of internet governanc...
9,GDC-input JBI,JBI,"Bioethics, Digital Ethics, Global Standards","Focuses on bioethics in the digital age, discu..."


#3-Preprocessing

In [6]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [21]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def preprocess(corpus):
    preprocessed = []
    for sentance in tqdm(corpus):
        #sentance = re.sub(r"http+", "", sentance)
        sentance = re.sub(r"http\S+", "", sentance)
        sentance = BeautifulSoup(sentance, 'lxml').get_text()
        sentance = decontracted(sentance)
        sentance = re.sub("\S*\d\S*", "", sentance).strip()
        sentance = re.sub('[^A-Za-z]+', ' ', sentance)
        sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
        preprocessed.append(sentance.strip())

    return preprocessed

In [23]:
df["Summary_transformed"] = preprocess(df['Summary'].values)

100%|██████████| 16/16 [00:00<00:00, 1033.08it/s]


#4-Clustering

In [45]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
import umap
import hdbscan
import pandas as pd

def preprocess_text(text):
    preprocessed_text = str(text).lower()  # Basic preprocessing, modify as needed
    return preprocessed_text

def process_batch(tokenizer, model, jobs):
    tokenized_articles = [tokenizer.encode_plus(
        preprocess_text(job),
        add_special_tokens=True,
        max_length=512,  # Define your desired max length
        padding='max_length',
        truncation=True,
        return_attention_mask=True,  # Include attention mask
        return_tensors='pt'
    ) for job in jobs]

    input_ids = torch.cat([article['input_ids'] for article in tokenized_articles], dim=0)
    attention_masks = torch.cat([article['attention_mask'] for article in tokenized_articles], dim=0)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_masks)
        embeddings = outputs[0][:, 0, :].cpu().numpy()  # Extract the [CLS] token embeddings, move to CPU

    return embeddings

def clustering(df, variable,batch_size=34):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    model.eval()

    embeddings_list = []
    for i in range(0, len(df), batch_size):
        print(i)
        batch_articles = df[variable].iloc[i:i+batch_size].tolist()
        batch_embeddings = process_batch(tokenizer, model, batch_articles)
        embeddings_list.append(batch_embeddings)

    embeddings = np.concatenate(embeddings_list, axis=0) if embeddings_list else np.array([])

    if embeddings.size > 0:
        reducer = umap.UMAP(n_components=5)
        reduced_embeddings = reducer.fit_transform(embeddings)

        clusterer = hdbscan.HDBSCAN(min_cluster_size=3)
        cluster_labels = clusterer.fit_predict(reduced_embeddings)

        df['cluster_type'] = cluster_labels
        df['embeddings'] = embeddings.tolist()

    return df


In [ ]:
clustering(df,"Summary_transformed")

In [50]:
df["Summary_transformed"] = df["Summary_transformed"].astype(str)
print('clustering agg',df['cluster_type'].value_counts())

clustering agg  1    9
 0    5
-1    2
Name: cluster_type, dtype: int64


#5-Topic Modelling

In [62]:
docs_per_topic_agg = df.groupby(['cluster_type'], as_index = False).agg({"Summary_transformed": ' '.join})

In [ ]:
docs_per_topic_agg

In [64]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

In [69]:
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=10):
    words = count.get_feature_names_out()
    labels = list(docs_per_topic.cluster_type)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['cluster_type'])
                     .text
                     .count()
                     .reset_index()
                     .rename({"Topic": "cluster", "Summary_transformed": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

In [71]:
tf_idf_agg, count_agg = c_tf_idf(docs_per_topic_agg.Summary_transformed.values, m=len(df))
top_n_words_agg = extract_top_n_words_per_topic(tf_idf_agg, count_agg, docs_per_topic_agg, n=10)
#topic_sizes_agg = extract_topic_sizes(df_train); topic_sizes_agg.head(10)

#6-Semantic Interpretation per Cluster

In [74]:
top_n_words_agg[-1]

[('stakeholders', 0.09241962407465937),
 ('education', 0.09241962407465937),
 ('governance', 0.09241962407465937),
 ('informed', 0.09241962407465937),
 ('addresses', 0.06931471805599453),
 ('inclusion', 0.06931471805599453),
 ('related', 0.06931471805599453),
 ('world', 0.05579921445238905),
 ('focusing', 0.05579921445238905),
 ('policy', 0.05579921445238905)]

In [75]:
top_n_words_agg[0]

[('advocating', 0.05134423559703298),
 ('strategies', 0.04133275144621411),
 ('role', 0.04133275144621411),
 ('access', 0.03632700937080467),
 ('rethinking', 0.034229490398021985),
 ('inequalities', 0.034229490398021985),
 ('information', 0.034229490398021985),
 ('innovative', 0.034229490398021985),
 ('learning', 0.034229490398021985),
 ('libraries', 0.034229490398021985)]

In [76]:
top_n_words_agg[1]

[('highlights', 0.02166084939249829),
 ('like', 0.019616911330918026),
 ('spyware', 0.019616911330918026),
 ('future', 0.019616911330918026),
 ('document', 0.019616911330918026),
 ('importance', 0.018174231403213763),
 ('development', 0.018174231403213763),
 ('focuses', 0.016245637044373717),
 ('ethical', 0.016245637044373717),
 ('principles', 0.016245637044373717)]

Cluster -1: Education and Governance

This cluster seems to revolve around themes of education, governance, and stakeholder engagement. The presence of words like 'stakeholders', 'education', 'governance', and 'informed' suggests a focus on the need for educating stakeholders about governance. The cluster likely represents documents that address the importance of informed participation in governance and the inclusion of diverse voices in policy-making processes.

Cluster 0: Advocacy and Information Access

The keywords 'advocating', 'strategies', 'role', 'access', 'learning', and 'libraries' point towards advocacy for information access and learning. The terms 'rethinking' and 'innovative' imply a call for new strategies or methods, potentially in the context of library services or educational resources. This cluster is likely about documents that discuss the strategic role of organizations in providing access to information and promoting continuous learning, with an emphasis on innovation and addressing inequalities.

Cluster 1: Technology Ethics and Regulation

With words such as 'spyware', 'future', 'ethical', and 'principles', this cluster seems focused on the ethical implications of technology and its future development. The term 'highlights' suggests these documents draw attention to specific issues like spyware and the need for ethical principles in technology development and use. 'Importance' and 'document' together indicate these might be official documents or position papers outlining the significance of ethical considerations in tech policy and development.

In [ ]:
df